# Getting routes

After testing out the API in notebook 1, this notebook gets data on travel times to Hattie Mae White on board meeting days. 

In [55]:
import numpy as np
import pandas as pd
import openpyxl
from fuzzywuzzy import process
from pandarallel import pandarallel
import requests
import time
import dateutil
from datetime import datetime, timedelta

# print all rows and columns when requested 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# initialize pandarallel

pandarallel.initialize()

# working directory

direc = "//Users//afan//Desktop//Misc//HMW_Transit//"

# Hattie Mae White coordinatess

HMW_lat = 29.802759908899148
HMW_long = -95.45410037006431

# API subscription key

api_key = "4e7902d1a2ae42df978fe20387049854"

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Bringing in school information 

In order to run this on all the schools, we need two pieces of information from two different datasets: The school coordinates, and the schools end times. Let's read in the two datasets first

In [58]:
# excel sheet with schools end times -- the information on all the schools is on the second sheet 
endtimes_str = direc + "prep//raw_data//" + "start_end_times.xlsx"
endtimes_cols = ['Campus Short Name', 'End Time']

endtimes = pd.read_excel(endtimes_str, 1, usecols = endtimes_cols, parse_dates = ['End Time'])

# delete SOAR Academy since there is no school end time 
endtimes = endtimes[endtimes['Campus Short Name'] != "SOAR Center"]
print(endtimes.shape)

endtimes.head()

(274, 2)


,Campus Short Name,End Time
0,Alcott ES,2022-01-06 15:00:00
1,Almeda ES,2022-01-06 15:00:00
2,Anderson ES,2022-01-06 15:00:00
3,Arabic Immersion,2022-01-06 15:00:00
4,Ashford ES,2022-01-06 15:00:00


In [59]:
# Geographic information 

coord_str =  direc + "prep//raw_data//" + "school_geography.csv"
coord_cols = ['School_Nam', 'X', 'Y']

coord = pd.read_csv(coord_str, usecols = coord_cols)
print(coord.shape)

coord.head()

(275, 3)


,X,Y,School_Nam
0,-95.331465,29.730194,AUSTIN H S
1,-95.469016,29.691198,BELLAIRE H S
2,-95.357984,29.782934,NORTHSIDE H S
3,-95.246987,29.778996,FURR H S
4,-95.339505,29.674387,JONES FUTURES ACADEMY


Since the school names are slightly different between these two datasets, I created a key to match the two. Let's read that in. 

In [60]:
key_str = direc + "prep//raw_data//" + "name_match_final.csv"
key = pd.read_csv(key_str)

key.head()

,Campus Short Name,School_Nam
0,Alcott ES,ALCOTT EL
1,Almeda ES,ALMEDA EL
2,Anderson ES,ANDERSON EL
3,Arabic Immersion,ARABIC IMMERSION MAGNET SCHOOL
4,Ashford ES,ASHFORD EL


Now, we can merge in the end time information with the coordinates information

In [61]:
endtime_coord = endtimes.merge(key).merge(coord)
endtime_coord.head()

,Campus Short Name,End Time,School_Nam,X,Y
0,Alcott ES,2022-01-06 15:00:00,ALCOTT EL,-95.329629,29.666614
1,Almeda ES,2022-01-06 15:00:00,ALMEDA EL,-95.419402,29.600185
2,Anderson ES,2022-01-06 15:00:00,ANDERSON EL,-95.487634,29.649295
3,Arabic Immersion,2022-01-06 15:00:00,ARABIC IMMERSION MAGNET SCHOOL,-95.388845,29.737199
4,Ashford ES,2022-01-06 15:00:00,ASHFORD EL,-95.598195,29.747265


In [72]:
# now, let's take a sample school and calculate the route! 
nrow = 2

# school name 
s_name = endtime_coord.iloc[nrow, 2]

# school end time
s_endtime = (endtime_coord.iloc[nrow, 1] + timedelta(hours=4)).strftime("%Y-%m-%dT%H:%M:%SZ")
print(s_endtime)

# For some reason, X and Y are longitude and latitude, respectively 
s_lat = endtime_coord.iloc[nrow, 4]
s_long = endtime_coord.iloc[nrow, 3]

print(s_lat, s_long)


2022-01-06T19:00:00Z
X   -95.487634
Y    29.649295
Name: 2, dtype: object
29.6492950384 -95.4876338111


In [73]:
trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&startTime=datetime'{s_endtime}'&$orderby=EndTime&$expand=Legs&subscription-key={api_key}")
   
print(trip.status_code)

200


Now we just gotta do some fun data wrangling 

In [74]:
trip_int = pd.DataFrame(trip.json())
trip_df = trip_int['value'].apply(pd.Series)
trip_df['school'] = endtime_coord.iloc[nrow, 2]
trip_df.head()

ItineraryId                       Created  \
0  60a2c532-a491-433c-b3a8-710d54b96358  2022-01-06T22:52:26.9131943Z   
1  a3da325b-f754-4e71-80a3-c19073785c7c  2022-01-06T22:52:26.9130072Z   
2  e2793c40-99bb-498f-8b05-4f36ab5d930f    2022-01-06T22:52:26.91306Z   
3  d4c0fb4c-e03c-443b-986a-65bd2642ca3c   2022-01-06T22:52:26.913392Z   

                                                                                                                            Kml  \
0  http://api.transitiq.com/ODataService.svc/GetItineraryKml?itineraryId=guid'60a2c532-a491-433c-b3a8-710d54b96358'&$format=kml   
1  http://api.transitiq.com/ODataService.svc/GetItineraryKml?itineraryId=guid'a3da325b-f754-4e71-80a3-c19073785c7c'&$format=kml   
2  http://api.transitiq.com/ODataService.svc/GetItineraryKml?itineraryId=guid'e2793c40-99bb-498f-8b05-4f36ab5d930f'&$format=kml   
3  http://api.transitiq.com/ODataService.svc/GetItineraryKml?itineraryId=guid'd4c0fb4c-e03c-443b-986a-65bd2642ca3c'&$format=kml   

              StartTime               EndTime     AdjustedStartTime  \
0  2022-01-06T23:04:57Z  2022-01-07T00:48:25Z  2022-01-06T23:04:57Z   
1  2022-01-06T23:47:27Z  2022-01-07T01:14:48Z  2022-01-06T23:47:27Z   
2  2022-01-06T23:47:27Z  2022-01-07T01:22:32Z  2022-01-06T23:47:27Z   
3  2022-01-06T23:17:35Z  2022-01-07T01:33:18Z  2022-01-06T23:17:35Z   

        AdjustedEndTime                 StartStopName  \
0  2022-01-07T00:48:25Z   W Airport Blvd @ Rampart St   
1  2022-01-07T01:14:48Z  Hillcroft Ave @ Ludington Dr   
2  2022-01-07T01:22:32Z  Hillcroft Ave @ Ludington Dr   
3  2022-01-07T01:33:18Z  Hillcroft Ave @ Ludington Dr   

                     EndStopName      StartStopId         EndStopId  \
0  W 18Th St @ Northwest Freeway  Ho414_4620_6285   Ho414_4620_8413   
1     Hempstead Rd @ Post Oak Rd  Ho414_4620_6337  Ho414_4620_11053   
2        Hempstead Rd @ W Loop N  Ho414_4620_6337   Ho414_4620_8896   
3  W 18Th St @ Northwest Freeway  Ho414_4620_6337   Ho414_4620_8413   

  StartAddress EndAddress  TransferCount TravelTypes  WalkDistance  \
0         None       None              3    Walk,Bus      0.642220   
1         None       None              2    Walk,Bus      0.819281   
2         None       None              2    Walk,Bus      1.201663   
3         None       None              3    Walk,Bus      0.596200   

                              RequestId   StartLon   StartLat   EndLon  \
0  73b043ca-404d-445b-b76b-1830b9e2184a -95.487634  29.649295 -95.4541   
1  73b043ca-404d-445b-b76b-1830b9e2184a -95.487634  29.649295 -95.4541   
2  73b043ca-404d-445b-b76b-1830b9e2184a -95.487634  29.649295 -95.4541   
3  73b043ca-404d-445b-b76b-1830b9e2184a -95.487634  29.649295 -95.4541   

     EndLat  LocalTimeOffset  \
0  29.80276               -5   
1  29.80276               -5   
2  29.80276               -5   
3  29.80276               -5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [48]:
legs = trip_df['Legs'].apply(pd.Series)
legs.head()

,0,1,2,3,4,5,6,7
0,"{'ItineraryLegId': 1884008878, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-06T22:36:08.4100318Z', 'StopName': '', 'StopId': '', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'DirectionName': '', 'RouteDestinationName': '', 'StartTime': '2022-01-06T22:46:28Z', 'AdjustedStartTime': '2022-01-06T22:46:28Z', 'Duration': 482, 'Length': 0.4154510810830502, 'Address': '', 'ShapeUri': '', 'TransportType': 'Walk', 'Ordinal': 0, 'ItineraryId': '988e6f6c-a283-451e-b328-c0cdbea77b6b', 'DataSource': None, 'Lat': 29.6492950384, 'Lon': -95.4876338111, 'TripId': '', 'TripHeadsign': '', 'DelayFromScheduleInSeconds': None}","{'ItineraryLegId': 1884008879, 'AgencyAbbreviation': 'RideMetro', 'AgencyName': 'Houston', 'Created': '2022-01-06T22:36:08.410064Z', 'StopName': 'Hillcroft Ave @ Ludington Dr', 'StopId': 'Ho414_4620_6337', 'RouteId': 'Ho414_4620_41920', 'RouteName': '047', 'DirectionId': 'Ho414_4620_41920_0', 'DirectionVariantId': 'Ho414_4620_41920_0_1', 'DirectionName': 'NORTHWEST TC', 'RouteDestinationName': 'NORTHWEST TC', 'StartTime': '2022-01-06T22:55:30Z', 'AdjustedStartTime': '2022-01-06T22:55:30Z', 'Duration': 3571, 'Length': 9.474549085974493, 'Address': '', 'ShapeUri': '', 'TransportType': 'Bus', 'Ordinal': 1, 'ItineraryId': '988e6f6c-a283-451e-b328-c0cdbea77b6b', 'DataSource': None, 'Lat': 29.649795, 'Lon': -95.492683, 'TripId': 'Ho414_4620_9266851', 'TripHeadsign': 'NORTHWEST TC', 'DelayFromScheduleInSeconds': 1.0}","{'ItineraryLegId': 1884008880, 'AgencyAbbreviation': 'RideMetro', 'AgencyName': 'Houston', 'Created': '2022-01-06T22:36:08.4100768Z', 'StopName': '79 - NORTHWEST BAY M', 'StopId': 'Ho414_4620_79_M', 'RouteId': 'Ho414_4620_41928', 'RouteName': '058', 'DirectionId': 'Ho414_4620_41928_1', 'DirectionVariantId': 'Ho414_4620_41928_1_0', 'DirectionName': 'WEST BELT', 'RouteDestinationName': 'WEST BELT', 'StartTime': '2022-01-07T00:00:00Z', 'AdjustedStartTime': '2022-01-07T00:00:00Z', 'Duration': 360, 'Length': 1.1043097343234265, 'Address': '', 'ShapeUri': '', 'TransportType': 'Bus', 'Ordinal': 3, 'ItineraryId': '988e6f6c-a283-451e-b328-c0cdbea77b6b', 'DataSource': None, 'Lat': 29.78307, 'Lon': -95.45546, 'TripId': 'Ho414_4620_9269229', 'TripHeadsign': 'WEST BELT', 'DelayFromScheduleInSeconds': 0.0}","{'ItineraryLegId': 1884008881, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-06T22:36:08.4100773Z', 'StopName': 'Hempstead Rd @ Post Oak Rd', 'StopId': 'Ho414_4620_11053', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'DirectionName': '', 'RouteDestinationName': '', 'StartTime': '2022-01-07T00:07:00Z', 'AdjustedStartTime': '2022-01-07T00:07:00Z', 'Duration': 468, 'Length': 0.40382963175920883, 'Address': '', 'ShapeUri': '', 'TransportType': 'Walk', 'Ordinal': 4, 'ItineraryId': '988e6f6c-a283-451e-b328-c0cdbea77b6b', 'DataSource': None, 'Lat': 29.799026, 'Lon': -95.456516, 'TripId': '', 'TripHeadsign': '', 'DelayFromScheduleInSeconds': None}","{'ItineraryLegId': 1884008882, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-06T22:36:08.4100773Z', 'StopName': '', 'StopId': '', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'DirectionName': '', 'RouteDestinationName': '', 'StartTime': '2022-01-07T00:07:00Z', 'AdjustedStartTime': '2022-01-07T00:07:00Z', 'Duration': 468, 'Length': 0.40382963175920883, 'Address': '', 'ShapeUri': '', 'TransportType': 'Arrived', 'Ordinal': 5, 'ItineraryId': '988e6f6c-a283-451e-b328-c0cdbea77b6b', 'DataSource': None, 'Lat': 29.802759908899148, 'Lon': -95.45410037006431, 'TripId': '', 'TripHeadsign': '', 'DelayFromScheduleInSeconds': None}",NaN,NaN,NaN
1,"{'ItineraryLegId': 1884009878, 'AgencyAbbreviation': None, 'AgencyName': None, 'Created': '2022-01-06T22:36:08.4101552Z', 'StopName': '', 'StopId': '', 'RouteId': '', 'RouteName': '', 'DirectionId': '', 'DirectionVariantId': '', 'DirectionName': '', 'RouteDestinationName': '', 'St

In [51]:
legs.iloc[0].apply(pd.Series)

,0,Address,AdjustedStartTime,AgencyAbbreviation,AgencyName,Created,DataSource,DelayFromScheduleInSeconds,DirectionId,DirectionName,DirectionVariantId,Duration,ItineraryId,ItineraryLegId,Lat,Length,Lon,Ordinal,RouteDestinationName,RouteId,RouteName,ShapeUri,StartTime,StopId,StopName,TransportType,TripHeadsign,TripId
0,NaN,,2022-01-06T22:46:28Z,None,None,2022-01-06T22:36:08.4100318Z,NaN,NaN,,,,482.0,988e6f6c-a283-451e-b328-c0cdbea77b6b,1.884009e+09,29.649295,0.415451,-95.487634,0.0,,,,,2022-01-06T22:46:28Z,,,Walk,,
1,NaN,,2022-01-06T22:55:30Z,RideMetro,Houston,2022-01-06T22:36:08.410064Z,NaN,1.0,Ho414_4620_41920_0,NORTHWEST TC,Ho414_4620_41920_0_1,3571.0,988e6f6c-a283-451e-b328-c0cdbea77b6b,1.884009e+09,29.649795,9.474549,-95.492683,1.0,NORTHWEST TC,Ho414_4620_41920,047,,2022-01-06T22:55:30Z,Ho414_4620_6337,Hillcroft Ave @ Ludington Dr,Bus,NORTHWEST TC,Ho414_4620_9266851
2,NaN,,2022-01-07T00:00:00Z,RideMetro,Houston,2022-01-06T22:36:08.4100768Z,NaN,0.0,Ho414_4620_41928_1,WEST BELT,Ho414_4620_41928_1_0,360.0,988e6f6c-a283-451e-b328-c0cdbea77b6b,1.884009e+09,29.783070,1.104310,-95.455460,3.0,WEST BELT,Ho414_4620_41928,058,,2022-01-07T00:00:00Z,Ho414_4620_79_M,79 - NORTHWEST BAY M,Bus,WEST BELT,Ho414_4620_9269229
3,NaN,,2022-01-07T00:07:00Z,None,None,2022-01-06T22:36:08.4100773Z,NaN,NaN,,,,468.0,988e6f6c-a283-451e-b328-c0cdbea77b6b,1.884009e+09,29.799026,0.403830,-95.456516,4.0,,,,,2022-01-07T00:07:00Z,Ho414_4620_11053,Hempstead Rd @ Post Oak Rd,Walk,,
4,NaN,,2022-01-07T00:07:00Z,None,None,2022-01-06T22:36:08.4100773Z,NaN,NaN,,,,468.0,988e6f6c-a283-451e-b328-c0cdbea77b6b,1.884009e+09,29.802760,0.403830,-95.454100,5.0,,,,,2022-01-07T00:07:00Z,,,Arrived,,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
schools.head(20)

## Running it on all the schools 

Now, let's loop through all the schools and put all the routes together 

In [ ]:
routelist = []

for nrow in range(0, len(schools)): 

  s_lat = schools.iloc[nrow, 1]
  s_long = schools.iloc[nrow, 2]

  # get the trip information for that school
  trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&$orderby=EndTime&$expand=Legs&subscription-key={key}")

  trip1 = pd.DataFrame(trip.json())

  trip_df = trip1['value'].apply(pd.Series)
  trip_df['school'] = schools.iloc[nrow, 0]
  trip_df['school_id'] = schools.iloc[nrow, 3]

  routelist.append(trip_df)
  print(f"Done with school #{nrow}: {schools.iloc[nrow, 0]}")

  time.sleep(2)   

In [ ]:
routes = pd.concat(routelist, ignore_index = True)
routes['AdjustedStartTime'] = routes.AdjustedStartTime.apply(dateutil.parser.isoparse)
routes['AdjustedEndTime'] = routes.AdjustedEndTime.apply(dateutil.parser.isoparse)
routes['length'] = routes['AdjustedEndTime'] - routes['AdjustedStartTime']

all_legs = routes['Legs'].apply(pd.Series)
leglist = []

for route in range(0, len(all_legs)): 

  temp = all_legs.iloc[route, :].apply(pd.Series)
  leglist.append(temp)

legs = pd.concat(leglist)
print(legs.shape)
legs = legs[~np.isnan(legs['Ordinal'])]
legs.shape

In [ ]:
print(f"Number of schools: {len(schools)}")
print(f"Number of routes: {len(routes)}")
print(f"Number of legs: {len(legs)}")

In [ ]:
legs.sort_values(['ItineraryId', 'Ordinal'], inplace = True)
legs.head()

In [ ]:
mintrans = routes[routes.TravelTypes != "Walk"].sort_values('TransferCount').drop_duplicates(subset = 'school_id', keep = 'first')
mintrans.TransferCount.value_counts()

In [ ]:
mintrans.length.describe()

In [ ]:
routes.length.describe()

In [ ]:
minlen = routes.sort_values('length').drop_duplicates(subset = 'school_id', keep = 'first')
minlen.length.describe()


In [ ]:
sum(minlen.length > datetime.timedelta(hours = 1))